In [1]:
import numpy as np

with open('variable_selection.npy', 'rb') as f:
    X = np.load(f)
    y = np.load(f)

In [2]:
## Imports
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

from sklearn.linear_model import Lars
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge,ElasticNet,Lasso
from sklearn.metrics import mean_squared_error

In [25]:
### Write your code here
class FeatureSelector:
    def __init__(self, Xtrain, Xvalid, Ytrain, Yvalid):
        self.Xtrain = Xtrain
        self.Xvalid = Xvalid
        self.Ytrain = Ytrain
        self.Yvalid = Yvalid
        self.m = Xtrain.shape[1]

    def score(self, V):
        Xt = np.concatenate([np.ones([self.Xtrain.shape[0], 1]), self.Xtrain[:, V]], axis=1)
        Xv = np.concatenate([np.ones([self.Xvalid.shape[0], 1]), self.Xvalid[:, V]], axis=1)
        
        # Initialize Least Angle Regression Model
        least_angle_regression = Lars()
        least_angle_regression.fit(Xt, self.Ytrain)
        
        pred_train = least_angle_regression.predict(Xt)
        pred_valid = least_angle_regression.predict(Xv)
        train_mse = ((pred_train - self.Ytrain) ** 2).mean()
        val_mse = ((pred_valid - self.Yvalid) ** 2).mean()
        return train_mse, val_mse

    def forward_search(self):
        improvement = True
        self.V_used = []
        self.V_left = list(np.arange(self.m))

        while improvement:
            # Calculate the current training and validation loss
            train_loss, val_loss = self.score(self.V_used)
            print()
            
          #  print("Training Loss: {}".format(train_loss))
           # print("Validation Loss: {}".format(val_loss))
            gain_best = 0

            # Iterate over the remaining features to find the one with the best gain
            for v in self.V_left:
                
                V_tmp = self.V_used.copy()
                V_tmp.append(v)
                print("self.Vused",self.V_used,"self.Vtemp",V_tmp)
                print(".score(.V_used)[1]",self.score(self.V_used)[1],"score(V_tmp)[1]",self.score(V_tmp)[1])
                gain = self.score(self.V_used)[1] - self.score(V_tmp)[1]

                if gain > gain_best:
                    gain_best = gain
                    v_best = v

            # Update selected features if there is a positive gain
            if gain_best > 0:
                self.V_used.append(v_best)
                self.V_left.remove(v_best)
            else:
                improvement = False

        # Return the selected features and final losses
        selected_features = self.V_used
        final_train_loss, final_val_loss = self.score(selected_features)
        return selected_features, final_train_loss, final_val_loss
    def backward_search(self):
        improvement = True
        self.V_used=list(np.arange(X_train.shape[1]))
        self.V_left=[]
        
        while improvement and len(self.V_used)>0:
            gain_best=0
            for v in self.V_used:
                V_tmp=self.V_used.copy()
                V_tmp.remove(v)
                gain=self.score(self.V_used)[1]-self.score(V_tmp)[1]
                if gain > gain_best:
                    gain_best=gain
                    v_best=v
            if gain_best>0:
                self.V_used.remove(v_best)
            else:
                improvement = False
        # Return the selected features and final losses
        selected_features = self.V_used
        final_train_loss, final_val_loss = self.score(selected_features)
        
        return selected_features, final_train_loss, final_val_loss
            
            

In [26]:
## Splitting the dataset
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)

In [27]:
## Testing Forward Search
feature_selector = FeatureSelector(X_train, X_valid, y_train, y_valid)

In [28]:
## Testing Backward Search
backward_selected_features, final_train_loss, final_val_loss = feature_selector.backward_search()